In [1]:
import numpy as np
import pandas as p
#for loading the datatset
from sklearn.datasets import fetch_20newsgroups
#library for importing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to C:\Users\Himanshu
[nltk_data]     Surana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#fetching the data
news_articles=fetch_20newsgroups()

In [3]:
#loading the data
x = news_articles.data
y = news_articles.target

In [5]:
#removing special characters from the data
def remove_special_char(x):
    special_char=['1','2','3','4','5','6','7','8','9','0','!','#','@','$','%','^','&','*','(',')',
                  '-','_','+','=','{','}','[',']',':',';','"',"'",'<',',','>','.','?','/','~','`',
                  '\n','\t']
    for i in special_char:
        x=x.replace(i,' ')
    return x

In [9]:
#removing the stop words
def filter_data(x):
    stop_word = set(stopwords.words("english"))
    stop_words_list = list(stop_word)
    new_x=[]
    for i in range(len(x)):
        filtered_words=[]
        temp=remove_special_char(x[i]).lower()
        words = temp.split(" ")
        for j in range(len(words)):
            if words[j] not in stop_words_list:
                filtered_words.append(words[j])
        temp_string = " "
        for k in filtered_words:
            temp_string+= k
            temp_string+=" "
        x[i].replace(x[i],temp_string)
        new_x.append(temp_string)            
    return new_x     
    
    
    

In [10]:
filtered_x=filter_data(x)

In [57]:
#creating the dataset
#finding all the different words in the data
setx=set()
list_of_words=[]
for i in range(len(filtered_x)):
    li=filtered_x[i].split()
    for i in li:
        list_of_words.append(i)
    temp_set=set(li)
    setx = setx.union(temp_set)



In [59]:
len(list_of_words)

2162623

In [60]:
len(setx)

91044

In [79]:
set_of_words=list(setx)
#finding the frequency of each word in the data sets 
frequency={}
for i in set_of_words:
    frequency[i]=list_of_words.count(i)

        
    

In [80]:
len(frequency)

91044

In [87]:
copy_frequency1=frequency.copy()

In [102]:
remaining_keys=copy_frequency1.keys()
low_freq_keys=[]
for i in remaining_keys:
    if copy_frequency1[i]<175:
        low_freq_keys.append(i)
print(len(low_freq_keys))

89064


In [103]:
#deleting the words having freq less than 175
for i in low_freq_keys:
    del copy_frequency1[i]
print(copy_frequency1)

{'short': 519, 'heart': 311, 'gay': 234, 'q\\': 483, 'xterm': 249, 'shown': 190, 'km': 248, 'security': 663, 'adobe': 181, 'ins': 231, 'cars': 474, 'old': 1214, 'stratus': 325, 'response': 408, 'greg': 251, 'independent': 237, 'flames': 205, 'dealer': 238, 'seem': 840, 'possibility': 192, 'cal': 204, 'local': 834, 'quickly': 201, 'radio': 525, 'piece': 195, 'floppy': 295, 'analysis': 279, 'goal': 362, 'compared': 215, 'supposed': 323, 'takes': 361, 'z': 4100, 'hey': 261, 'never': 1806, 'theory': 400, 'jews': 868, 'cause': 651, 'compatible': 204, 'approach': 291, 'correct': 574, 'along': 433, 'realize': 262, 'friend': 457, 'started': 624, 'played': 271, 'means': 940, 'game': 1192, 'despite': 175, 'march': 267, 'facts': 229, 'science': 1394, 'shot': 341, 'communications': 516, 'authority': 315, 'sciences': 211, 'eff': 229, 'existence': 376, 'rlk': 237, 'j': 3845, 'moment': 202, 'virginia': 569, 'supply': 300, 'makes': 826, 'using': 1899, 'apr': 3954, 'brought': 297, 'mit': 999, 'faster':

In [117]:
vocabulary=list(copy_frequency1.keys())

In [134]:
#makinng the x train dataset
x_data=np.array([[0 for i in range(len(copy_frequency1))] for j in range(len(news_articles.data))])
x_data.shape

(11314, 1980)

In [135]:
for i in range(len(filtered_x)):
    for j in range(len(vocabulary)):
        if vocabulary[j] in filtered_x[i].split():
            x_data[i,j]+=1
print(x_data)        

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [143]:
x_data.shape

(11314, 1980)

In [154]:
y_data=news_articles.target

In [158]:
#performing train and test split on the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,random_state=0)

In [159]:
#applying multinomial Naive Bayes from Sklearn
from sklearn.naive_bayes import MultinomialNB
clf1=MultinomialNB()
clf1.fit(x_train,y_train)
y_pred1=clf1.predict(x_test)

In [160]:
clf1.score(x_test,y_test)

0.807705903145988

In [288]:
#applying multinomial naive bayes from scratch
import math as m
def fit(x_train,y_train,vocabulary):
    result={}
    #top level keys will have the differtent values of the classes
    class_values=set(y_train)
    result["total_data"]=len(y_train)
    for current_class in class_values:
        result[current_class]={}
        num_words=x_train.shape[1]
        #accessing rows with class equals to current class
        current_class_rows=(y_train==current_class)
        x_train_current=x_train[current_class_rows]
        y_train_current=y_train[current_class_rows]
        result[current_class]["total_count"]=0
        for j in vocabulary:
            result[current_class][j]=x_train_current[:,vocabulary.index(j)].sum()
            result[current_class]["total_count"]+=result[current_class][j]
    return result

In [292]:
def probablity(dictionary,x,current_class):
    output=np.log(dictionary[current_class]["total_count"])-np.log(dictionary["total_data"])
    num_words=len(dictionary[current_class].keys())-2
    for i in vocabulary:
        if x[vocabulary.index(i)]==0:
            prob_wi_current_class=0
        else:
            prob_wi_current_class=(np.log(dictionary[current_class][i]+1))-(np.log(dictionary[current_class]["total_count"]+num_words))
        output+=prob_wi_current_class
    return output

In [293]:
def predictsinglepoint(dictionary,x):
    classes=dictionary.keys()
    best_p = 0
    best_class = 0
    first = True
    for current_class in classes:
        if current_class == "total_data":
            continue
        p_current_class = probablity(dictionary,x,current_class)
        if first or p_current_class > best_p :
            best_p = p_current_class
            best_class = current_class
            first=False
    return best_class

In [294]:
def predict(dictionary,x_test):
    y_pred=[]
    for x in x_test:
        x_class = predictsinglepoint(dictionary,x)
        y_pred.append(x_class)
    return y_pred

In [289]:
dictionary=fit(x_train,y_train,vocabulary)

In [295]:
y_pred2=predict(dictionary,x_test)

In [296]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       112
           1       0.58      0.72      0.64       147
           2       0.72      0.71      0.72       140
           3       0.66      0.68      0.67       148
           4       0.75      0.71      0.73       149
           5       0.81      0.81      0.81       159
           6       0.70      0.89      0.78       131
           7       0.82      0.77      0.79       158
           8       0.87      0.85      0.86       162
           9       0.83      0.85      0.84       148
          10       0.93      0.92      0.93       150
          11       0.97      0.90      0.93       155
          12       0.74      0.73      0.74       147
          13       0.88      0.84      0.86       131
          14       0.88      0.83      0.85       154
          15       0.89      0.87      0.88       155
          16       0.90      0.89      0.90       144
          17       0.94    